In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score  
from sklearn.metrics import classification_report
from sklearn.metrics import cohen_kappa_score
import pickle
from xgboost import XGBClassifier


In [ ]:
rela_embedding=pd.read_csv('../data/new_Embedding_Model_POLISH_D128-rela.csv').drop('Unnamed: 0',axis=1)
tail_embedding=pd.read_csv('../data/tail_embedding_POLISH_D128.csv').drop('Unnamed: 0',axis=1)
x_test = pd.read_csv("../data/5year-Ts.csv")
polish_train_val=pd.read_csv("../data/5year-Tr.csv")
y_train_val=polish_train_val['LABEL']
y_test=x_test['LABEL']
x_test=x_test.drop(['LABEL','Company'],axis=1)
polish_train_val=polish_train_val.drop(['LABEL','Company'],axis=1)
x_test=x_test.replace(' ', np.nan).astype(float)
polish_train_val=polish_train_val.replace(' ', np.nan).astype(float)
x_trainval_mapping_df=pd.read_csv('../data/x_trainval_map_tailv.csv').drop('Unnamed: 0',axis=1)
x_test_mapping_df=pd.read_csv('../data/x_test_map_tailv.csv').drop('Unnamed: 0',axis=1)

In [ ]:
train_data=pd.read_csv('../data/train_data.csv')
val_data=pd.read_csv('../data/val_data.csv')
train_y=pd.read_csv('../data/train_y.csv')
val_y=pd.read_csv('../data/val_y.csv')


In [ ]:

# define a function to construct the desired string
def format_value(col, val):
    if pd.isna(val):
        return val
    else:
        return col + ':' + str(val)

x_train_mapping=train_data.copy()
x_train_dct = x_train_mapping.to_dict(orient='list')
x_train_mapping_dct = {}
for col in x_train_dct:
    x_train_mapping_dct[col] = [format_value(col, val) for val in x_train_dct[col]]

x_train_mapping=train_data.copy()
x_train_dct = x_train_mapping.to_dict(orient='list')
x_train_mapping_dct = {}
for col in x_train_dct:
    x_train_mapping_dct[col] = [format_value(col, val) for val in x_train_dct[col]]

x_val_mapping=val_data.copy()
x_val_dct = x_val_mapping.to_dict(orient='list')
x_val_mapping_dct = {}
for col in x_val_dct:
    x_val_mapping_dct[col] = [format_value(col, val) for val in x_val_dct[col]]

x_val_mapping_df=pd.DataFrame(x_val_mapping_dct)
x_train_mapping_df=pd.DataFrame(x_train_mapping_dct)
x_val_mapping_df.to_csv('../data/x_val_mapping_df.csv')
x_train_mapping_df.to_csv('../data/x_train_mapping_df.csv')


In [ ]:
test_notnull=[]
for i in range(len(x_test_mapping_df)):
    total_test_reltal=[]
    for j in x_test_mapping_df.columns:
        tailv=tail_embedding[tail_embedding['tail']==x_test_mapping_df.loc[i,j]].drop(['tail'],axis=1).values.tolist()
        if tailv:
          total_test_reltal.append((tailv[0]))
        else:
          total_test_reltal.append([0]*256)

    test_notnull.append(total_test_reltal)
      

test_notnull_len=[]
for i in range(len(test_notnull)):
    test_notnull_len.append(len(test_notnull[i]))

filename = '../data/polish_test_onlytail.pickle'
# open the file for writing in binary mode
with open(filename, 'wb') as f:
    # serialize the list and write it to the file
    pickle.dump(test_notnull, f)

In [ ]:
train_notnull=[]
for i in range(len(x_train_mapping_df)):
    total_train_reltal=[]
    for j in x_test_mapping_df.columns:
        tailv=tail_embedding[tail_embedding['tail']==x_test_mapping_df.loc[i,j]].drop(['tail'],axis=1).values.tolist()
        if tailv:
          total_test_reltal.append((tailv[0]))
        else:
          total_test_reltal.append([0]*256)

    test_notnull.append(total_test_reltal)
      

test_notnull_len=[]
for i in range(len(test_notnull)):
    test_notnull_len.append(len(test_notnull[i]))

filename = '../data/polish_test_onlytail.pickle'
# open the file for writing in binary mode
with open(filename, 'wb') as f:
    # serialize the list and write it to the file
    pickle.dump(test_notnull, f)

In [ ]:
train_notnull=[]
for i in range(len(x_train_mapping_df)):
    total_train_reltal=[]
    for j in x_train_mapping_df.columns:
        tailv=tail_embedding[tail_embedding['tail']==x_train_mapping_df.loc[i,j]].drop(['tail'],axis=1).values.tolist()
        if tailv:
          total_train_reltal.append((tailv[0]))
        else:
          total_train_reltal.append([0]*256)

    train_notnull.append(total_train_reltal)
      

train_notnull_len=[]
for i in range(len(train_notnull)):
    train_notnull_len.append(len(train_notnull[i]))

filename = '../data/polish_train_onlytail.pickle'
# open the file for writing in binary mode
with open(filename, 'wb') as f:
    # serialize the list and write it to the file
    pickle.dump(train_notnull, f)



In [ ]:
val_notnull=[]
for i in range(len(x_val_mapping_df)):
    total_val_reltal=[]
    for j in x_val_mapping_df.columns:
        tailv=tail_embedding[tail_embedding['tail']==x_val_mapping_df.loc[i,j]].drop(['tail'],axis=1).values.tolist()
        if tailv:
          total_val_reltal.append((tailv[0]))
        else:
          total_val_reltal.append([0]*256) # it should be 128 since it just has tails, replacement in the below code box

    val_notnull.append(total_val_reltal)
      

val_notnull_len=[]
for i in range(len(val_notnull)):
    val_notnull_len.append(len(val_notnull[i]))

filename = '../data/polish_val_onlytail.pickle'
# open the file for writing in binary mode
with open(filename, 'wb') as f:
    # serialize the list and write it to the file
    pickle.dump(val_notnull, f)




In [ ]:
# Define replacement dictionary with tuple keys
replace_dict = {tuple([0]*256): [0]*128}

# Replace vectors using dictionary mapping, then convert back to lists
val_notnull = [[replace_dict.get(tuple(item), item) for item in sublist] for sublist in val_notnull]
filename = '../data/polish_val_onlytail.pickle'
# open the file for writing in binary mode
with open(filename, 'wb') as f:
    # serialize the list and write it to the file
    pickle.dump(val_notnull, f)



In [ ]:
# Define replacement dictionary with tuple keys
replace_dict = {tuple([0]*256): [0]*128}

# Replace vectors using dictionary mapping, then convert back to lists
train_notnull = [[replace_dict.get(tuple(item), item) for item in sublist] for sublist in train_notnull]
filename = '../data/polish_train_onlytail.pickle'
# open the file for writing in binary mode
with open(filename, 'wb') as f:
    # serialize the list and write it to the file
    pickle.dump(train_notnull, f)



In [ ]:
# Define replacement dictionary with tuple keys
replace_dict = {tuple([0]*256): [0]*128}

# Replace vectors using dictionary mapping, then convert back to lists
test_notnull = [[replace_dict.get(tuple(item), item) for item in sublist] for sublist in test_notnull]
filename = '../data/polish_test_onlytail.pickle'
# open the file for writing in binary mode
with open(filename, 'wb') as f:
    # serialize the list and write it to the file
    pickle.dump(test_notnull, f)



In [ ]:
print(len(test_notnull))
print(len(test_notnull[0]))
print(len(test_notnull[0][0]))
print(len(train_notnull[0]))


1182
64
128
64


in order to input train_data and val_data into xgboost(2-D data), flatten train_data and val_data


In [ ]:
train_flatten = np.reshape(train_notnull, (len(train_notnull), 64*128))
val_flatten=np.reshape(val_notnull, (len(val_notnull), 64*128))
test_flatten = np.reshape(test_notnull, (len(test_notnull), 64*128))
print(train_flatten.shape)
print(val_flatten.shape)
print(test_flatten.shape)


(4018, 8192)
(710, 8192)
(1182, 8192)


In [ ]:
from xgboost import XGBClassifier

model = XGBClassifier(max_depth=8)
model.fit(train_flatten, train_y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=8, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [ ]:

y_predict=model.predict_proba(val_flatten)[:, 1]

#find the optimal threshold
precision, recall, thresholds = precision_recall_curve(val_y, y_predict)

df_recall_precision = pd.DataFrame({'Precision':precision[:-1],
                                    'Recall':recall[:-1],
                                    'Threshold':thresholds})


np.seterr(divide='ignore', invalid='ignore')
f1_score = (2 * precision * recall) / (precision + recall)
findex=list(f1_score).index(max(f1_score))
thresholdOpt = round(thresholds[findex], ndigits = 4)
fscoreOpt = round(f1_score[findex], ndigits = 4)
recallOpt = round(recall[findex], ndigits = 4)
precisionOpt = round(precision[findex], ndigits = 4)
print('Best Threshold: {} , F-Score: {}'.format(thresholdOpt, fscoreOpt))
print('Recall: {}, Precision: {}'.format(recallOpt, precisionOpt))

from sklearn.metrics import precision_score, classification_report, recall_score, f1_score, roc_auc_score, confusion_matrix, accuracy_score
y_pre_test=model.predict_proba(test_flatten)[:, 1]
defaulter_decision = (y_pre_test >= thresholdOpt)
print(classification_report(y_test, defaulter_decision))

f1_test = f1_score(y_test, defaulter_decision, average='macro') # use 'micro' or 'weighted' for multi-class problems depending on the problem
print(f'test dataset F1 Score: {f1_test}')

Best Threshold: 0.19529999792575836 , F-Score: 0.7586
Recall: 0.6735, Precision: 0.8684
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      1095
           1       0.90      0.62      0.73        87

    accuracy                           0.97      1182
   macro avg       0.94      0.81      0.86      1182
weighted avg       0.97      0.97      0.96      1182

test dataset F1 Score: 0.8585512689514236
